# Seleccionar idioma

In [1]:
idioma = 'english'

# Importar Librerias

In [2]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2020-09-25


In [3]:
import nltk; 
#nltk.download('stopwords')
import time


In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from string import digits
from string import punctuation
import unidecode


In [5]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [6]:
import spacy


In [7]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [9]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words(idioma)
stop_words.extend(['linkremoved','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])


In [11]:
LdaModel = gensim.models.ldamodel.LdaModel


### Import data

In [12]:
region = "northamerica"

In [13]:
#file_name = 'english_europe_tweets_20190411.csv'
file_name = 'english_northamerica_tweets_20190411.csv'




In [14]:
df  = pd.read_csv('../../data/cambridge_analytica/regional_datasets/'+file_name)

In [15]:
df['text'] = df['texto_completo']

In [16]:
df['text'].replace({"http\\S+": '<linkremoved>'}, inplace=True, regex=True)
df['text'].replace({"<link removed>": '<linkremoved>'}, inplace=True, regex=True)
for texto in df['text'][0:5]:
    print(texto)


<usernameremoved> Hold Facebook accountable for abusing our data and censoring our voice. Vote no on the CRA. <linkremoved>
Both sides should just drop it. Trump won...elections over. Move on. <linkremoved>
$AAPL lobbing threes while they can. Wasnt it just a few years back when their platform was actually hacked and people private photos were exposed? <linkremoved>
15 min #RSI Signals:

$BTC - $SLR: 6.84
$BTC - $OMNI: 6.97
$BTC - $RBY: 6.99
$BTC - $SPHR: 21.13
$BTC - $RDN: 21.17
$BTC - $MCO: 21.48
$BTC - $VIBE: 23.76
$BTC - $ADX: 26.25
$BTC - $PPT: 26.63

#Bitcoin #tokensale #smartcontract #ETH #DAPP $LTC #signals #RADS #BigData #litecoin
<usernameremoved> Hold Facebook accountable for abusing our data and censoring our voice. Vote no on the CRA.

<linkremoved>


In [17]:
df['text'].replace({"@[^\\s]+": '<usernameremoved>'}, inplace=True, regex=True)
df.drop_duplicates(['text'],keep='first', inplace=True)

In [18]:
punctuation+='¡¿'

In [19]:
#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

In [20]:
import unidecode

data = []
for tweet in df['text']:
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    tweet = unidecode.unidecode(tweet)
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    data.append(tweet)


In [21]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(tknzr.tokenize(sentence))


In [22]:
data_words = list(sent_to_words(data))


In [23]:
# Build the bigram and trigram models
#quizas el min count es muy bajo
#documentaicon de bigramas 
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#3importpackages
min_count = int(len(df)*0.02)
bigram = gensim.models.Phrases(data_words, min_count=min_count, threshold=1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], min_count =min_count, threshold=1)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

['both', 'sides', 'should', 'just', 'drop', 'it', 'trump', 'won', 'elections', 'over', 'move', 'on', 'linkremoved']


In [24]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [25]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)


In [27]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)



In [28]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
!python -m spacy download en

C:\Users\gonza\tesisenv\Scripts\python.exe: No module named spacy


In [29]:

nlp = spacy.load('en', disable=['parser', 'ner'])

In [30]:

#sera mejor ocupar wordnetlemattizer?
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote']]


In [31]:
#The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]




In [32]:
texts

[['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote'],
 ['side', 'drop', 'trump', 'election', 'move'],
 ['lob',
  'three',
  'year',
  'back',
  'platform',
  'actually',
  'hack',
  'people',
  'private',
  'photo',
  'expose'],
 ['signal', 'rad', 'bigdata'],
 ['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote'],
 ['lmfao'],
 ['user', 'lock', 'horse', 'go'],
 ['worried', 'ethic', 'lol'],
 ['ship', 'even', 'trust', 'much', 'get', 'tired'],
 ['sell', 'dietary', 'habit', 'know', 'eat', 'frozen'],
 ['delete', 'poison', 'datum', 'first'],
 ['pay',
  'behavioral',
  'microtargete',
  'psychographic',
  'messaging',
  'electiondata'],
 ['reveal', 'real', 'turd', 'fear'],
 ['thank',
  'broadcast',
  'exactly',
  'have',
  'thinking',
  'know',
  'facebook',
  'whole',
  'business',
  'model',
  'mine',
  'sell'],
 ['good', 'morning'],
 ['finally',
  'article',
  'demystify',
  'machine',
  'learning',
  'target',
  'model',
  'really',
  'work'],
 ['icymi',

In [33]:
#corpus
import pickle
with open(region+'_texts.pkl', 'wb') as f:
    pickle.dump(texts, f)

### Import lda model

In [34]:
ldamallet = LdaModel.load("../../data/cambridge_analytica/regional_datasets/files_"+region+"/english_"+region+"_tweets_20190411.csv_gensim.model")


# Keywords by topic - colocar numero de topicos a extraer

In [35]:
num_topicos_actual = -1

In [36]:
topicos = ldamallet.show_topics(formatted=False, num_words=100, num_topics=num_topicos_actual)
list_of_df = list()
for topic in topicos:
    number, words_weight = topic
    words_weight_df=pd.DataFrame(words_weight, columns=['term', 'weight'])
    list_of_df.append(words_weight_df)

In [37]:
len(topicos)

11

In [38]:
keywords_by_topic= pd.concat(list_of_df,axis=1)
keywords_by_topic

,term,weight,term,weight,term,weight,term,weight,term,weight,...,term,weight,term,weight,term,weight,term,weight,term,weight
0,facebook,0.070277,time,0.031174,bigdata,0.068912,amp,0.034804,datum,0.180956,...,privacy,0.105992,people,0.093064,work,0.043842,trump,0.136736,question,0.047166
1,delete,0.041229,back,0.023785,business,0.029554,leave,0.020844,user,0.072966,...,facebook,0.030867,news,0.031912,ad,0.038408,retweete,0.018169,today,0.039066
2,year,0.036485,put,0.018099,big,0.027626,love,0.020365,give,0.043599,...,change,0.026170,lie,0.023978,election,0.029196,family,0.015741,medium,0.038429
3,post,0.032074,man,0.014620,analytic,0.021793,free,0.019484,company,0.038966,...,issue,0.017608,story,0.018132,pay,0.028080,call,0.015268,social,0.037642
4,twitter,0.031258,show,0.014390,learn,0.018677,conservative,0.019186,data,0.033719,...,protect,0.015842,fake,0.016929,campaign,0.027169,child,0.013218,watch,0.037127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,verify,0.001494,mad,0.002365,skill,0.002185,threat,0.002011,partnership,0.001300,...,dataprivacy,0.002142,corporation,0.002133,hacker,0.002114,enemy,0.002063,touch,0.001744
96,artist,0.001456,personally,0.002306,job,0.002185,determine,0.001972,monetize,0.001226,...,monitor,0.002132,complain,0.002079,relevant,0.002114,guilty,0.002040,episode,0.001735
97,memory,0.001441,car,0.002301,increase,0.002167,letter,0.001967,agency,0.001217,...,condition,0.002113,shirt,0.002064,cash,0.002090,sander,0.001997,stage,0.001710
98,screen,0.001408,sex,0.002276,performance,0.002092,alternative,0.001962,potentially,0.001157,...,ensure,0.002061,fakenew,0.002054,process,0.002013,member,0.001955,camera,0.001696


In [39]:
len(keywords_by_topic)

100

In [40]:
keywords_by_topic.to_csv(region+'_'+str(today)+'_'+'keywords_weights.csv', encoding='utf-8')

# Highest topic per tweet

In [41]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic['Dominant_Topic'] = df_dominant_topic['Dominant_Topic'].astype(int)



In [42]:
df_dominant_topic.to_csv(region+'_'+str(today)+'_'+'highest_topic_per_tweet.csv', encoding='utf-8', index=False)

#  Find the most representative tweets for each topic

In [43]:
# Group top 5 sentences under each topic
numero_de_tweets_por_topico = 50
sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic') # esta matriz contiene todos los tweets con el porcentaje de contribucion y max topico. se agrupa y se obtiene lo q queremos

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(numero_de_tweets_por_topico)], 
                                            axis=0)


# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

sent_topics_sorteddf_mallet['Topic_Num'] = sent_topics_sorteddf_mallet['Topic_Num'].astype(int)

# Show
sent_topics_sorteddf_mallet


,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.2864,"facebook, delete, year, post, twitter, account...",years it took for each product to reach mil...
1,0,0.2688,"facebook, delete, year, post, twitter, account...",hi jay \n\nwe have received a request to perma...
2,0,0.2661,"facebook, delete, year, post, twitter, account...",usernameremoved my facebook privacy are set ...
3,0,0.2619,"facebook, delete, year, post, twitter, account...",facebook delete his text message thread and a...
4,0,0.2612,"facebook, delete, year, post, twitter, account...",may twitter please show instagram photos pict...
...,...,...,...,...
545,10,0.2284,"question, today, medium, social, watch, live, ...",facebook execs have been on mps hot seat for ...
546,10,0.2273,"question, today, medium, social, watch, live, ...",out of senators were active facebook use...
547,10,0.2272,"question, today, medium, social, watch, live, ...",extra seats have been added in the capitol hil...
548,10,0.2268,"question, today, medium, social, watch, live, ...",amid missteps zuckerberg faces congressional ...


In [44]:

sent_topics_sorteddf_mallet.to_csv(region+'_'+str(today)+'_'+'most_representative_tweets_per_topic.csv', encoding='utf-8', index=False)

# Topic distribution across documents

Muestra el numero de documentos y el porcentaje de documentos en donde ese topico es el principal componente. 
El código siguiente fue arreglado. El del tutorial tenía errores.

In [45]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords.drop_duplicates(subset='Dominant_Topic', keep='first').sort_values(by=['Dominant_Topic']).reset_index(drop=True), topic_counts.sort_index(), topic_contribution.sort_index()], axis=1)

# Change Column names

df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

df_dominant_topics['Dominant_Topic'] = df_dominant_topics['Dominant_Topic'].astype(int)


In [46]:
#funciona perfecto
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,0,"facebook, delete, year, post, twitter, account...",48789,0.1425
1.0,1,"time, back, put, man, show, start, turn, guy, ...",29016,0.0847
2.0,2,"bigdata, business, big, analytic, learn, tech,...",35198,0.1028
3.0,3,"amp, leave, love, free, conservative, call, gr...",25312,0.0739
4.0,4,"datum, user, give, company, data, information,...",30742,0.0898
5.0,5,"make, good, thing, read, bad, great, time, fee...",27499,0.0803
6.0,6,"privacy, facebook, change, issue, protect, law...",28094,0.0821
7.0,7,"people, news, lie, story, fake, care, report, ...",25119,0.0734
8.0,8,"work, ad, election, pay, campaign, money, vote...",29070,0.0849
9.0,9,"trump, retweete, family, call, child, break, p...",29596,0.0864


In [47]:
df_dominant_topics.to_csv(region+'_'+str(today)+'_'+'topic_distribution_across_documents.csv', encoding='utf-8', index=False)

# Visualize topics with Mallet

In [111]:
#prueba = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)


In [112]:
pyLDAvis.enable_notebook()


In [113]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus,id2word, mds='pcoa', sort_topics=False, R=30)


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


In [148]:
pyLDAvis.display(vis)

In [146]:
pyLDAvis.save_html(vis,region+"_"+str(today)+"_visualization.html")

In [147]:
pyLDAvis.save_json(vis,region+"_"+str(today)+"_visualization.json")

In [338]:
#para cargar json model
#esto no funciona, hay que arreglasrlo.

import json
def load_R_model(filename):
    with open(filename, 'r') as j:
        data_input = json.load(j)
    data = {'topic_term_dists': data_input['phi'], 
            'doc_topic_dists': data_input['theta'],
            'doc_lengths': data_input['doc.length'],
            'vocab': data_input['vocab'],
            'term_frequency': data_input['term.frequency']}
    return data

def vis_R_model(filename):
    return pyLDAvis.display(pyLDAvis.prepare(**load_R_model(filename)))